In [47]:
import tqdm
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
from sklearn import svm
import matplotlib.pyplot as plt

%matplotlib inline

In [87]:
df = pd.read_csv('../data/preprocessed.csv')

In [88]:
df[:5]

,page_name,contributors_count_ar,contributors_count_avg_all_period,contributors_count_avg_last_month,contributors_count_avg_last_week,contributors_count_max,contributors_count_mean,contributors_count_min,contributors_count_std,revisions_count_ar,revisions_count_avg_all_period,revisions_count_avg_last_month,revisions_count_avg_last_week,revisions_count_max,revisions_count_mean,revisions_count_min,revisions_count_std
0,Himantolophus_borealis,0.0,1.0,1.0,1.0,1,1.0,1,0.0,0.0,5.0,5.0,5.0,5,5.0,5,0.0
1,Himantolophus_brevirostris,0.0,2.0,2.0,2.0,2,2.0,2,0.0,0.0,6.0,6.0,6.0,6,6.0,6,0.0
2,Himantolophus_compressus,0.0,3.0,3.0,3.0,3,3.0,3,0.0,0.0,7.0,7.0,7.0,7,7.0,7,0.0
3,Himantolophus_cornifer,0.0,2.0,2.0,2.0,2,2.0,2,0.0,0.0,9.0,9.0,9.0,9,9.0,9,0.0
4,Himantolophus_crinitus,0.0,2.0,2.0,2.0,2,2.0,2,0.0,0.0,4.0,4.0,4.0,4,4.0,4,0.0


In [89]:
df.columns.tolist()

['page_name',
 'contributors_count_ar',
 'contributors_count_avg_all_period',
 'contributors_count_avg_last_month',
 'contributors_count_avg_last_week',
 'contributors_count_max',
 'contributors_count_mean',
 'contributors_count_min',
 'contributors_count_std',
 'revisions_count_ar',
 'revisions_count_avg_all_period',
 'revisions_count_avg_last_month',
 'revisions_count_avg_last_week',
 'revisions_count_max',
 'revisions_count_mean',
 'revisions_count_min',
 'revisions_count_std']

Check standard deviations to check whether we have features with ero std.

In [102]:
df[1:].std()

TypeError: unhashable type: 'slice'

### Correlational map

In [106]:
# plt.figure
# plt.matshow(df[1:].corr())
# scatter_matrix(df[1:9], alpha=0.2, figsize=(8, 8), diagonal='kde') #gives singular matrix error
df.drop("page_name", axis=1).corr()

,contributors_count_ar,contributors_count_avg_all_period,contributors_count_avg_last_month,contributors_count_avg_last_week,contributors_count_max,contributors_count_mean,contributors_count_min,contributors_count_std,revisions_count_ar,revisions_count_avg_all_period,revisions_count_avg_last_month,revisions_count_avg_last_week,revisions_count_max,revisions_count_mean,revisions_count_min,revisions_count_std
contributors_count_ar,1.000000,0.361500,0.363886,0.371090,0.371437,0.361500,0.353943,0.993147,0.999018,0.132848,0.133513,0.135559,0.136995,0.132848,0.130456,0.998104
contributors_count_avg_all_period,0.361500,1.000000,0.999997,0.999946,0.999940,1.000000,0.999964,0.276715,0.329937,0.684045,0.684201,0.684698,0.687216,0.684045,0.682884,0.404158
contributors_count_avg_last_month,0.363886,0.999997,1.000000,0.999969,0.999964,0.999997,0.999940,0.279185,0.332359,0.683777,0.683934,0.684436,0.686955,0.683777,0.682611,0.406488
contributors_count_avg_last_week,0.371090,0.999946,0.999969,1.000000,0.999992,0.999946,0.999830,0.286699,0.339692,0.683079,0.683242,0.683760,0.686277,0.683079,0.681898,0.413489
contributors_count_max,0.371437,0.999940,0.999964,0.999992,1.000000,0.999940,0.999811,0.286747,0.339926,0.682199,0.682361,0.682880,0.685412,0.682199,0.681013,0.413992
contributors_count_mean,0.361500,1.000000,0.999997,0.999946,0.999940,1.000000,0.999964,0.276715,0.329937,0.684045,0.684201,0.684698,0.687216,0.684045,0.682884,0.404158
contributors_count_min,0.353943,0.999964,0.999940,0.999830,0.999811,0.999964,1.000000,0.269157,0.322367,0.685555,0.685705,0.686186,0.688690,0.685555,0.684413,0.396644
contributors_count_std,0.993147,0.276715,0.279185,0.286699,0.286747,0.276715,0.269157,1.000000,0.997350,0.087462,0.088133,0.090191,0.091163,0.087462,0.085182,0.984071
revisions_count_ar,0.999018,0.329937,0.332359,0.339692,0.339926,0.329937,0.322367,0.997350,1.000000,0.115854,0.116522,0.118576,0.119839,0.115854,0.113501,0.994396
revisions_count_avg_all_period,0.132848,0.684045,0.683777,0.683079,0.682199,0.684045,0.685555,0.087462,0.115854,1.000000,1.000000,0.999996,0.999983,1.000000,0.999997,0.156017


In [107]:
df_array = np.array(df.drop("page_name", axis=1))

In [108]:
train_ratio = 0.7
train_size = int(train_ratio * df_array.shape[0])
# exclude page_name
X_train = df_array[:train_size,:]
X_test = df_array[train_size:,:]
# Generate some abnormal novel observations
# X_outliers = np.random.uniform(low=-4, high=4, size=(20, 2))

In [109]:
# source: http://scikit-learn.org/stable/auto_examples/svm/plot_oneclass.html 

# fit the model
#TODO: add outliers!
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
clf.fit(X_train)
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
# y_pred_outliers = clf.predict(X_outliers)
n_error_train = y_pred_train[y_pred_train == -1].size
n_error_test = y_pred_test[y_pred_test == -1].size
# n_error_outliers = y_pred_outliers[y_pred_outliers == 1].size
train_accuracy = n_error_train/y_pred_train.size
test_accuracy = n_error_test/y_pred_test.size

In [110]:
print("Train acc: {0}\nTest acc: {1}".format(train_accuracy, test_accuracy))

Train acc: 0.4857142857142857
Test acc: 0.6333333333333333
